In [4]:
import os
import time
import requests
import pandas as pd
import random
from urllib.parse import urlencode
from fake_useragent import UserAgent

In [5]:
class BuffCsgo:
    def __init__(self, cateogry, save_file_path, _, price_range):
        self._ = _
        self.category = cateogry
        self.save_file_path = save_file_path
        self.price_range = price_range
        self.item_datas = []
        self.base_url = "https://buff.163.com/api/market/goods?"
    
    def get_current_time(self):
        return round(time.time()*1000)
    
    #Get total number of pages
    def get_total_page(self):
        params = {
            "game": "csgo",
            "page_num": 1,
            "category_group": self.category,
            "min_price": 50,
            "max_price": 50 + self.price_range,
            "_": self._
        }
        url = self.base_url + urlencode(params)
        try:
            response = requests.get(url=url, headers=self.init_headers(), proxies=self.random_ip(), timeout=10)
            if response.status_code == 200:
                page_text = response.json()
                if page_text.get("data"):
                    total_page = page_text.get("data").get("total_page")
                    return total_page
        except requests.ConnectionError as e:
            print("Connection Error")
    
    #Getting contents of the pages
    def get_page(self):
        for page in range(1,self.get_total_page()+1):
            params = {
                "game": "csgo",
                "page_num": page,
                "category_group": self.category,
                "min_price": 50,
                "max_price": 50 + self.price_range,
                "_": self._
            }
            current_url = self.base_url + urlencode(params)
            try:
                response = requests.get(url=current_url, headers=self.init_headers(), proxies=self.random_ip(), timeout=10)
                if response.status_code == 200:
                    print(f"Obtained page {page}")
                    self._ = self.get_current_time()
                    page_text = response.json()
                    self.parse_page(page_text)
                    self.save_to_csv()
                    time.sleep(random.random() * 8)
            except requests.ConnectionError as e:
                print("Failed")
                
    #Obtaining specific details
    def parse_page(self, page_text):
        if page_text.get("data").get("items"):
            for item in page_text.get("data").get("items"):
                info = {}
                info["饰品名称"] = item.get("name")
                info["Buff Current Price"] = item.get("quick_price")
                info["Number of Current Sales on Buff"] = item.get("sell_num")
                self.item_datas.append(info)
    
    #Cookies and stuff
    def init_headers(self):
        cookie = "Device-Id=1dWAd0cD7jxpMmfEZznm; Locale-Supported=en; game=csgo; NTES_YD_SESS=BFAFcXa3P4rClnlcojaBJtOWZKr6Y9wXFOEy8neUuuTstMe.tG1Vzr2ev3jsE_pukCoc4Glcp3KE6NnfDbSuPJ0eKMR_ELHfCDRFKtKOdaMZSlnOJAmWkG8lR80OCDyzSB1.cA7PuzcScMRfGtNoPDqGRrm4YupTvGsW1qYN1yLbd82aSJVwSFNIjD0wUqw4lZbMOtd8GEyXSX5ZWIlfNsISBTlLm7S1lLuBf2xeq0egJ; S_INFO=1740019757|0|0&60##|18121007305; P_INFO=18121007305|1740019757|1|netease_buff|00&99|null&null&null#IN&null#10#0|&0||18121007305; remember_me=U1084335450|aTeTwRaP8keAw66wLdiEgOyYTo8IsMht; session=1-AeoWZdCCaEA18HMUgkGC3CVVaagu2HwFWC_n07gFQxpb2020555266; csrf_token=IjZkMzFiOGJkY2RlZDYzYzZlY2IxOTI5OWM1MGUyYWVjODUxOGQwYjUi.Z7apHg.bME3qdeuQhLHtVrDH48DF0dqmrI"
        headers = {
            "User-Agent": UserAgent().random,
            "Cookie": cookie
        }
        return headers
    
    def random_ip(self):
        proxies = [
            '120.232.150.110:80',
            '106.45.221.69:3256',
            '47.98.208.18:8080',
            '117.24.80.59:3256',
            '111.179.73.203:3256',
            '47.95.178.212:3128',
            '125.87.84.82:3256',
            '47.98.179.39:8080',
            '116.62.113.142:1081',
            '114.215.172.136:31280',
            '47.98.183.59:3128',
            '118.194.242.184:80',
            '114.67.108.243:8081',
            '120.232.150.100:80'
        ]
        proxy = {
            "http": "http://" + random.choice(proxies)
        }
        return proxy
    
    #Save to csv
    def save_to_csv(self):
        df = pd.DataFrame(self.item_datas)
        df = df.reindex(columns=["饰品名称", "Buff Current Price", "Number of Current Sales on Buff"])
        if os.path.exists(self.save_file_path) and os.path.getsize(self.save_file_path):
            df.to_csv(self.save_file_path, mode="a", encoding="utf-8", index=False)
        else:
            df.to_csv(self.save_file_path, mode='a', encoding='utf-8', index=False)
            print("Created" + self.save_file_path)
        self.item_datas = []
      
    

In [ ]:
for category in ["knive", "glove", "smg","shotgun", "machinegun"]:
    save_file_path = "{}.csv".format(category)
    B = BuffCsgo(category, save_file_path, 1614323440986, 200)
    total_page = B.get_total_page()
    print(f"Current category {category} has a total of {total_page} pages")
    B.get_page()

Current category shotgun has a total of None pages


TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'